# ⏱️ Early Stopping: Knowing When to Stop Training

This notebook demonstrates **Early Stopping**—a simple yet effective regularization technique that stops training when the model starts to overfit. We'll compare training with and without early stopping on the Sonar dataset.

## What You'll Learn

1. Why training too long leads to overfitting
2. How early stopping works
3. Implementing early stopping with patience
4. Saving and restoring the best model weights

---

## 1. Setup and Imports

In [ ]:
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

## 2. Loading the Sonar Dataset

The **Sonar dataset** is ideal for demonstrating early stopping because:
- Small size (208 samples) makes overfitting likely
- High dimensionality (60 features) increases model complexity
- Binary classification task is easy to evaluate

In [ ]:
df = pd.read_csv("sonar.all-data", header=None)
print(f"Dataset shape: {df.shape}")
print(f"\nClass distribution:")
print(df[60].value_counts())

## 3. Data Preprocessing

In [ ]:
# Convert labels to numeric
df[60] = df[60].map({'M': 0, 'R': 1})

# Split features and target
X = df.drop(60, axis=1)
y = df[60]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=1
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

---

## 4. Model Definition

We'll use a simple feedforward network for this demonstration.

In [ ]:
class SimpleNN(nn.Module):
    """Simple neural network for binary classification."""
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(60, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.network(x)

## 5. Training Without Early Stopping

First, let's train for a fixed number of epochs (20) and observe what happens.

### The Overfitting Problem

```
Epoch 1-5:   Model learns useful patterns (both losses decrease)
Epoch 6-10:  Model reaches optimal point (validation loss minimum)
Epoch 11-20: Model overfits (train loss ↓, validation loss ↑)
```

Without early stopping, we train past the optimal point!

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    """Standard training without early stopping."""
    train_losses, val_losses, val_accuracies = [], [], []
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        y_pred, y_true = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                y_pred.extend(predicted.cpu().numpy())
                y_true.extend(labels.cpu().numpy())
        
        val_losses.append(val_loss / len(val_loader))
        val_accuracy = accuracy_score(y_true, y_pred)
        val_accuracies.append(val_accuracy)
        
        print(f"Epoch {epoch+1:2d}/{epochs} | "
              f"Train Loss: {train_losses[-1]:.4f} | "
              f"Val Loss: {val_losses[-1]:.4f} | "
              f"Val Acc: {val_accuracy:.4f}")
    
    return train_losses, val_losses, val_accuracies

In [ ]:
# Train without early stopping
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Training WITHOUT Early Stopping (20 epochs)")
print("=" * 60)
train_losses, val_losses, val_accuracies = train_model(
    model, train_loader, test_loader, criterion, optimizer, epochs=20
)

In [ ]:
# Visualize training progress
plt.figure(figsize=(10, 6))
epochs_range = range(1, 21)

plt.plot(epochs_range, val_accuracies, 'g-o', label="Validation Accuracy", linewidth=2)
plt.axhline(y=max(val_accuracies), color='r', linestyle='--', 
            label=f"Best Accuracy: {max(val_accuracies):.4f}")

# Mark the best epoch
best_epoch = val_accuracies.index(max(val_accuracies)) + 1
plt.axvline(x=best_epoch, color='orange', linestyle='--', 
            label=f"Best Epoch: {best_epoch}")

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Validation Accuracy', fontsize=12)
plt.title('Training Without Early Stopping', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nBest validation accuracy was at epoch {best_epoch}")
print(f"But we trained until epoch 20!")

---

## 6. Training With Early Stopping

Now let's implement early stopping to automatically stop at the optimal point.

### How Early Stopping Works

```
1. Track validation metric (accuracy or loss)
2. If metric improves:
   - Save model weights (checkpoint)
   - Reset patience counter
3. If metric doesn't improve:
   - Increment patience counter
4. If patience exceeded:
   - Stop training
   - Restore best weights
```

### Key Parameter: Patience

**Patience** = number of epochs to wait for improvement before stopping

- `patience=3`: Stop quickly, may miss better solutions
- `patience=10`: More exploration, longer training
- `patience=5-7`: Good balance for most cases

In [ ]:
def train_model_early_stop(model, train_loader, val_loader, criterion, optimizer, 
                           epochs=20, patience=3):
    """
    Training with early stopping.
    
    Args:
        patience: Number of epochs to wait for improvement
    """
    best_accuracy = 0
    counter = 0
    best_epoch = 0
    
    train_losses, val_losses, val_accuracies = [], [], []
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        y_pred, y_true = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                y_pred.extend(predicted.cpu().numpy())
                y_true.extend(labels.cpu().numpy())
        
        val_losses.append(val_loss / len(val_loader))
        val_accuracy = accuracy_score(y_true, y_pred)
        val_accuracies.append(val_accuracy)
        
        print(f"Epoch {epoch+1:2d}/{epochs} | "
              f"Train Loss: {train_losses[-1]:.4f} | "
              f"Val Loss: {val_losses[-1]:.4f} | "
              f"Val Acc: {val_accuracy:.4f}", end="")
        
        # Early stopping logic
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_epoch = epoch + 1
            counter = 0
            # Save best model
            torch.save(model.state_dict(), "best_model.pt")
            print(" ← Best model saved!")
        else:
            counter += 1
            print(f" (patience: {counter}/{patience})")
            if counter >= patience:
                print(f"\n🛑 Early Stopping triggered at epoch {epoch+1}!")
                print(f"   Best accuracy was {best_accuracy:.4f} at epoch {best_epoch}")
                break
    
    return train_losses, val_losses, val_accuracies, best_epoch

In [ ]:
# Train with early stopping
model_es = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_es.parameters(), lr=0.001)

print("Training WITH Early Stopping (patience=3)")
print("=" * 60)
train_losses_es, val_losses_es, val_accuracies_es, best_epoch = train_model_early_stop(
    model_es, train_loader, test_loader, criterion, optimizer, 
    epochs=20, patience=3
)

In [ ]:
# Visualize early stopping
plt.figure(figsize=(10, 6))
epochs_range = range(1, len(val_accuracies_es) + 1)

plt.plot(epochs_range, val_accuracies_es, 'g-o', label="Validation Accuracy", linewidth=2)
plt.axvline(x=best_epoch, color='red', linestyle='--', linewidth=2,
            label=f"Best Epoch: {best_epoch}")
plt.axvline(x=len(val_accuracies_es), color='orange', linestyle='--', linewidth=2,
            label=f"Stopped at Epoch: {len(val_accuracies_es)}")

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Validation Accuracy', fontsize=12)
plt.title('Training WITH Early Stopping', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

---

## 7. Restoring the Best Model

Early stopping saved the best model weights. Let's load them and verify the performance.

In [ ]:
# Load the best model weights
model_es.load_state_dict(torch.load("best_model.pt"))
print("✓ Loaded best model weights")

In [ ]:
# Evaluate the restored model
model_es.eval()
y_pred, y_true = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_es(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

final_accuracy = accuracy_score(y_true, y_pred)
print(f"\nFinal Test Accuracy (Best Model): {final_accuracy:.4f}")

---

## 8. Comparison Summary

In [ ]:
print("\n" + "=" * 50)
print("COMPARISON SUMMARY")
print("=" * 50)

print(f"\nWithout Early Stopping:")
print(f"  Epochs trained: 20")
print(f"  Best accuracy: {max(val_accuracies):.4f} (at epoch {val_accuracies.index(max(val_accuracies))+1})")
print(f"  Final accuracy: {val_accuracies[-1]:.4f}")

print(f"\nWith Early Stopping (patience=3):")
print(f"  Epochs trained: {len(val_accuracies_es)}")
print(f"  Best accuracy: {max(val_accuracies_es):.4f} (at epoch {best_epoch})")
print(f"  Final accuracy (restored): {final_accuracy:.4f}")

print(f"\n💡 Early stopping saved {20 - len(val_accuracies_es)} epochs of training!")

---

## 9. Key Takeaways

### Benefits of Early Stopping

| Benefit | Description |
|---------|-------------|
| **Prevents overfitting** | Stops before model memorizes training data |
| **Saves computation** | No wasted epochs after optimal point |
| **Simple to implement** | Just track validation metric |
| **No hyperparameters to tune** | Only patience (easy to set) |

### Best Practices

1. **Always save checkpoints** of the best model
2. **Use patience of 5-10** for most cases
3. **Monitor validation loss OR accuracy** (not training metrics)
4. **Restore best weights** after stopping
5. **Combine with other regularization** (dropout, L2) for best results

### When to Use Early Stopping

- Almost always! It's a safe, effective technique
- Especially important for small datasets
- When training time is a concern
- When you're not sure how many epochs to train